<a href="https://colab.research.google.com/github/rwcitek/C11-capstone-project/blob/main/eBird/ebird_bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with BigQuery


We'll see if this runs in CoLab.




### Set up the workspace


In [ ]:
!rm -rf data/


In [ ]:
mkdir data

In [ ]:
cd data

/content/data


In [ ]:
pwd

'/content/data'

### Download some sample data

In [ ]:
%%capture output
%%bash
apt-get update
apt-get install -y tree jq curl less

In [ ]:
!curl -O 'https://ebird.org/downloads/samples/ebd-datafile-SAMPLE.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  455k  100  455k    0     0   618k      0 --:--:-- --:--:-- --:--:--  619k


In [ ]:
!ls -la


total 468
drwxr-xr-x 2 root root   4096 Nov 22 13:43 .
drwxr-xr-x 1 root root   4096 Nov 22 13:43 ..
-rw-r--r-- 1 root root 466849 Nov 22 13:43 ebd-datafile-SAMPLE.zip


In [ ]:
!unzip ebd-datafile-SAMPLE.zip

Archive:  ebd-datafile-SAMPLE.zip
   creating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/
  inflating: __MACOSX/._ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._ebd_US-AL-101_202204_202204_relApr-2022.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/terms_of_use.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._terms_of_use.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/BCRCodes.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._BCRCodes.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/eBird_Basic_Dataset_Metadata_v1.14.pdf  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._eBird_Basic_Dataset_Metadata_v1.14.pdf  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/IBACod

In [ ]:
!ls -la


total 476
drwxr-xr-x 4 root root   4096 Nov 22 13:43 .
drwxr-xr-x 1 root root   4096 Nov 22 13:43 ..
-rw-r--r-- 1 root root 466849 Nov 22 13:43 ebd-datafile-SAMPLE.zip
drwxr-xr-x 2 root root   4096 Jun  8  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE
drwxr-xr-x 3 root root   4096 Nov 22 13:43 __MACOSX


In [ ]:
!rm -rf __MACOSX/

In [ ]:
!tree ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/


ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/
├── BCRCodes.txt
├── ebd_US-AL-101_202204_202204_relApr-2022.txt
├── eBird_Basic_Dataset_Metadata_v1.14.pdf
├── IBACodes.txt
├── recommended_citation.txt
├── terms_of_use.txt
└── USFWSCodes.txt

0 directories, 7 files


In [ ]:
%%bash
wc -l ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


1400 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


## Set up Google Cloud Platform

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [41]:
import os
project_id = "default-256400"
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id
os.environ["GOOGLE_CLOUD_BUCKET"] = "c11-ebird"


In [3]:
project_id

'default-256400'

In [4]:
os.environ["GOOGLE_CLOUD_PROJECT"]

'default-256400'

## Need to create a *bucket*

Go to https://console.cloud.google.com/storage/browser




Got a bucket:

- https://console.cloud.google.com/storage/browser/c11-ebird;tab=objects?project=default-256400&prefix=&forceOnObjectsSortingFiltering=false

## Phase 1:



1. upload the CSV file to GS
1. import into BQ



In [ ]:
file = 'ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt'


In [ ]:
!gcloud storage cp {file} gs://c11-ebird/

Copying file://ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt


In [ ]:
!gcloud storage ls -l gs://c11-ebird/

    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T00:57:52Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz
TOTAL: 2 objects, 564709 bytes (551.47kiB)


### From web interface

Create a data set and import CSV to data set:

- https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

### Repeat using bq command

In [5]:
!bq ls {project_id}:ebird

          tableId            Type    Labels   Time Partitioning   Clustered Fields  
 -------------------------- ------- -------- ------------------- ------------------ 
  ebd_NM_relSep_2023         TABLE                                                  
  ebd_sampling_relSep_2023   TABLE                                                  
  ebird_small                TABLE                                                  


In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_2

In [6]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_2 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt


Waiting on bqjob_r41bf882ca6d188fa_0000018bf8187ee7_1 ... (0s) Current status: DONE   


In [7]:
!bq ls {project_id}:ebird

          tableId            Type    Labels   Time Partitioning   Clustered Fields  
 -------------------------- ------- -------- ------------------- ------------------ 
  ebd_NM_relSep_2023         TABLE                                                  
  ebd_sampling_relSep_2023   TABLE                                                  
  ebird_small                TABLE                                                  
  ebird_small_2              TABLE                                                  


In [8]:
!bq show {project_id}:ebird.ebird_small_2

Table default-256400:ebird.ebird_small_2

   Last modified                   Schema                   Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- --------------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  22 Nov 17:33:58   |- GLOBAL_UNIQUE_IDENTIFIER: string     1399         534727                                                            534727                                               
                    |- LAST_EDITED_DATE: timestamp                                                                                                                                              
                    |- TAXONOMIC_ORDER: integer                                                                                                                                           

In [11]:
!bq --project_id=${GOOGLE_CLOUD_PROJECT} query "select * from ebird.ebird_small_2 limit 10"

+-------------------------------------------------+---------------------+-----------------+----------+------------------+------------------+----------------------+------------------------+----------------------------+-------------+-------------------+---------------+-------------------+---------------+---------+---------------+--------------+---------+------------+------------+-------------+----------+----------+------------+-------------+------------------------------------------------------------+-------------+---------------+-----------+-------------+------------------+---------------------------+-------------+---------------------------+---------------+---------------+--------------+------------------+--------------------+----------------+------------------+----------------------+------------------+-----------+----------+----------+--------+---------------+-------------------------------------------------------------------------------------------------------------------------------

In [17]:
%%bigquery df --project {project_id}
select *
from ebird.ebird_small_2
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

In [18]:
df

,GLOBAL_UNIQUE_IDENTIFIER,LAST_EDITED_DATE,TAXONOMIC_ORDER,CATEGORY,TAXON_CONCEPT_ID,COMMON_NAME,SCIENTIFIC_NAME,SUBSPECIES_COMMON_NAME,SUBSPECIES_SCIENTIFIC_NAME,EXOTIC_CODE,...,NUMBER_OBSERVERS,ALL_SPECIES_REPORTED,GROUP_IDENTIFIER,HAS_MEDIA,APPROVED,REVIEWED,REASON,TRIP_COMMENTS,SPECIES_COMMENTS,string_field_49
0,URN:CornellLabOfOrnithology:EBIRD:OBS1380605933,2022-04-02 09:55:59.208261+00:00,20496,species,avibase-1DA430B8,Blue Jay,Cyanocitta cristata,None,None,<NA>,...,1,1,None,0,1,0,None,None,None,None
1,URN:CornellLabOfOrnithology:EBIRD:OBS1380605935,2022-04-02 09:55:59.212963+00:00,8773,species,avibase-7AA076EF,Barred Owl,Strix varia,None,None,<NA>,...,1,1,None,0,1,0,None,None,None,None
2,URN:CornellLabOfOrnithology:EBIRD:OBS1380605934,2022-04-02 09:55:59.209422+00:00,1180,species,avibase-9C5ED06A,Wild Turkey,Meleagris gallopavo,None,None,<NA>,...,1,1,None,0,1,0,None,None,None,None
3,URN:CornellLabOfOrnithology:EBIRD:OBS1380605936,2022-04-02 09:55:59.214060+00:00,27403,species,avibase-8E1D9327,Wood Thrush,Hylocichla mustelina,None,None,<NA>,...,1,1,None,0,1,0,None,None,None,None
4,URN:CornellLabOfOrnithology:EBIRD:OBS1380683993,2022-04-02 11:12:42.368739+00:00,7909,species,avibase-EB98812F,Cooper's Hawk,Accipiter cooperii,None,None,<NA>,...,1,0,None,0,1,0,None,None,None,None
5,URN:CornellLabOfOrnithology:EBIRD:OBS1380953818,2022-04-02 14:41:30.486432+00:00,31267,species,avibase-B4EE123D,Purple Finch,Haemorhous purpureus,None,None,<NA>,...,1,0,None,0,1,0,None,None,Male,None
6,URN:CornellLabOfOrnithology:EBIRD:OBS1380974027,2022-04-02 14:59:14.096444+00:00,7412,species,avibase-4FF7DE80,Black Vulture,Coragyps atratus,None,None,<NA>,...,1,0,None,0,1,0,None,None,None,None
7,URN:CornellLabOfOrnithology:EBIRD:OBS1381139222,2022-04-02 17:46:33.220130+00:00,31905,species,avibase-37E9CCDA,Chipping Sparrow,Spizella passerina,None,None,<NA>,...,1,0,None,0,1,0,None,None,None,None
8,URN:CornellLabOfOrnithology:EBIRD:OBS1383013775,2022-04-04 14:55:28.984346+00:00,27389,species,avibase-B01E8BD4,Hermit Thrush,Catharus guttatus,None,None,<NA>,...,2,0,None,0,1,0,None,None,None,None
9,URN:CornellLabOfOrnithology:EBIRD:OBS1386285800,2022-04-08 15:53:44.167690+00:00,1180,species,avibase-9C5ED06A,Wild Turkey,Meleagris gallopavo,None,None,<NA>,...,5,0,None,0,1,0,None,None,Tall brown bird of classic turkey silhouette a...,None


## Phase 2:


1. compress the CSV file with gzip
1. upload the compressed CSV file to GS
1. import into BQ



In [ ]:
!ls -la {file}

-rw-r--r-- 1 root root 504698 May 19  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


In [ ]:
!gzip {file}

In [ ]:
!ls -la {file}.gz


-rw-r--r-- 1 root root 60011 May 19  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


In [ ]:
!gcloud storage cp {file}.gz gs://c11-ebird/

Copying file://ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


In [21]:
!gcloud storage ls -l gs://c11-ebird/

1032617810  2023-11-22T14:27:03Z  gs://c11-ebird/data.nm.txt.gz
    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
27103974002  2023-11-22T08:07:44Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt
4815040900  2023-11-22T07:20:39Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
TOTAL: 4 objects, 32952137410 bytes (30.69GiB)


In [19]:
!bq ls {project_id}:ebird

          tableId            Type    Labels   Time Partitioning   Clustered Fields  
 -------------------------- ------- -------- ------------------- ------------------ 
  ebd_NM_relSep_2023         TABLE                                                  
  ebd_sampling_relSep_2023   TABLE                                                  
  ebird_small                TABLE                                                  
  ebird_small_2              TABLE                                                  


In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_3

In [20]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_3 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


BigQuery error in load operation: Error processing job
'default-256400:bqjob_r6fc80c33e33569d1_0000018bf824e293_1': Not found: URI
gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


Waiting on bqjob_r6fc80c33e33569d1_0000018bf824e293_1 ... (0s) Current status: DONE   


CalledProcessError: ignored

In [ ]:
!bq ls {project_id}:ebird

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  ebird_small     TABLE                                                  
  ebird_small_3   TABLE                                                  


In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_3

In [ ]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


## Phase 3:


1. compress the CSV file with gzip
1. create a tar file with the compressed CSV file
1. untar and stream the compressed CSV file to GS
1. import into BQ



In [ ]:
!ls -la {file}.gz

-rw-r--r-- 1 root root 60011 May 19  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


Create the tar file

In [ ]:
path, file1 = file.split("/")

In [ ]:
!tar -cvf example.tar -C {path} {file1}.gz

ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


In [ ]:
!ls -la


total 544
drwxr-xr-x 3 root root   4096 Nov 22 06:37 .
drwxr-xr-x 1 root root   4096 Nov 22 06:22 ..
-rw-r--r-- 1 root root 466849 Nov 22 06:23 ebd-datafile-SAMPLE.zip
drwxr-xr-x 2 root root   4096 Nov 22 06:25 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE
-rw-r--r-- 1 root root  71680 Nov 22 06:44 example.tar


In [ ]:
!tar -tvf example.tar

-rw-r--r-- root/root     60011 2022-05-19 17:35 ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


Untar and stream to GS.

In [ ]:
!gcloud storage ls -l gs://c11-ebird/


    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T00:57:52Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz
TOTAL: 2 objects, 564709 bytes (551.47kiB)


In [ ]:
%%bash
tar -O -xf example.tar ebd_US-AL-101_202204_202204_relApr-2022.txt.gz |
gcloud storage cp - gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz



Copying file://- to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz
  
......


In [23]:
!gcloud storage ls -l gs://c11-ebird/

1032617810  2023-11-22T14:27:03Z  gs://c11-ebird/data.nm.txt.gz
    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
27103974002  2023-11-22T08:07:44Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt
4815040900  2023-11-22T07:20:39Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
TOTAL: 4 objects, 32952137410 bytes (30.69GiB)


In [22]:
!bq ls {project_id}:ebird

          tableId            Type    Labels   Time Partitioning   Clustered Fields  
 -------------------------- ------- -------- ------------------- ------------------ 
  ebd_NM_relSep_2023         TABLE                                                  
  ebd_sampling_relSep_2023   TABLE                                                  
  ebird_small                TABLE                                                  
  ebird_small_2              TABLE                                                  


In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_4

In [ ]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_4 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz


Waiting on bqjob_r80dce339e5bf06_0000018bf5cd9540_1 ... (1s) Current status: DONE   


In [ ]:
!bq ls {project_id}:ebird

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  ebird_small     TABLE                                                  
  ebird_small_4   TABLE                                                  


In [ ]:
!bq show {project_id}:ebird.ebird_small

In [ ]:
!bq show {project_id}:ebird.ebird_small_4

In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_4

## Phase 4:



1. compress the CSV file with gzip
1. create a tar file with the compressed CSV file
1. upload the tar file to DO's object store
1. get, untar, and stream the compressed CSV file to GS
1. import into BQ


In [24]:
!curl -I https://cnmi-ebird.sfo3.digitaloceanspaces.com/example.tar

HTTP/2 200 
content-length: 71680
accept-ranges: bytes
last-modified: Wed, 22 Nov 2023 06:55:56 GMT
x-rgw-object-type: Normal
etag: "fb5a9186a43fc4d150feb7bfff027363"
x-amz-request-id: tx00000bee062f4ff82cacc-00655e3f9a-3c6f48c0-sfo3a
content-type: application/x-tar
date: Wed, 22 Nov 2023 17:51:22 GMT
vary: Origin, Access-Control-Request-Headers, Access-Control-Request-Method
strict-transport-security: max-age=15552000; includeSubDomains; preload
x-envoy-upstream-healthchecked-cluster: 



In [25]:
%%bash
curl -s https://cnmi-ebird.sfo3.digitaloceanspaces.com/example.tar |
tar -O -xf - ebd_US-AL-101_202204_202204_relApr-2022.txt.gz |
dd bs=1M status=progress |
gcloud storage cp - gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz


0+5 records in
0+5 records out
60011 bytes (60 kB, 59 KiB) copied, 0.309062 s, 194 kB/s
Copying file://- to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz
  
....


11100700672 bytes (11 GB, 10 GiB) copied, 25 s, 444 MB/s
0+1751059 records in
0+1751059 records out
11479859200 bytes (11 GB, 11 GiB) copied, 25.8381 s, 444 MB/s
^C


In [26]:
!gcloud storage ls -l gs://c11-ebird/

1032617810  2023-11-22T14:27:03Z  gs://c11-ebird/data.nm.txt.gz
    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T17:52:45Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz
27103974002  2023-11-22T08:07:44Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt
4815040900  2023-11-22T07:20:39Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
TOTAL: 5 objects, 32952197421 bytes (30.69GiB)


In [28]:
!curl -I https://cnmi-ebird.sfo3.digitaloceanspaces.com/ebd_relSep-2023.tar

HTTP/2 200 
content-length: 200955054080
accept-ranges: bytes
last-modified: Thu, 16 Nov 2023 17:18:22 GMT
x-rgw-object-type: Normal
etag: "61420aff1e69e47d679e92353ab0332f-913"
x-amz-meta-s3cmd-attrs: atime:1699546641/ctime:1699889200/gid:0/gname:root/md5:086921907fba6ecdf715782309a3e5d0/mode:33188/mtime:1699672342/uid:0/uname:root
x-amz-request-id: tx0000002a239604640fa19-00655e4059-3c6f4933-sfo3a
content-type: application/x-tar
date: Wed, 22 Nov 2023 17:54:33 GMT
vary: Origin, Access-Control-Request-Headers, Access-Control-Request-Method
strict-transport-security: max-age=15552000; includeSubDomains; preload
x-envoy-upstream-healthchecked-cluster: 



In [ ]:
# 200,955,054,080

In [27]:
!ls -la

total 24
drwxr-xr-x 1 root root 4096 Nov 20 14:42 .
drwxr-xr-x 1 root root 4096 Nov 22 17:24 ..
drwxr-xr-x 1 root root 4096 Nov 22 17:31 .config
drwxr-xr-x 1 root root 4096 Nov 20 14:42 sample_data


In [29]:
!bq ls {project_id}:ebird

          tableId            Type    Labels   Time Partitioning   Clustered Fields  
 -------------------------- ------- -------- ------------------- ------------------ 
  ebd_NM_relSep_2023         TABLE                                                  
  ebd_sampling_relSep_2023   TABLE                                                  
  ebird_small                TABLE                                                  
  ebird_small_2              TABLE                                                  


In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_5

In [30]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_5 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz


Waiting on bqjob_r2512cc9f16e3a39b_0000018bf82e5fcc_1 ... (1s) Current status: DONE   


In [31]:
!bq ls {project_id}:ebird

          tableId            Type    Labels   Time Partitioning   Clustered Fields  
 -------------------------- ------- -------- ------------------- ------------------ 
  ebd_NM_relSep_2023         TABLE                                                  
  ebd_sampling_relSep_2023   TABLE                                                  
  ebird_small                TABLE                                                  
  ebird_small_2              TABLE                                                  
  ebird_small_5              TABLE                                                  


In [33]:
!bq show {project_id}:ebird.ebird_small

Table default-256400:ebird.ebird_small

   Last modified                   Schema                   Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- --------------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  21 Nov 23:41:57   |- GLOBAL_UNIQUE_IDENTIFIER: string     1399         534727                                                            534727                41452                          
                    |- LAST_EDITED_DATE: timestamp                                                                                                                                              
                    |- TAXONOMIC_ORDER: integer                                                                                                                                             

In [ ]:
!bq show {project_id}:ebird.ebird_small_5

Table default-256400:ebird.ebird_small_5

   Last modified                   Schema                   Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- --------------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  22 Nov 07:06:00   |- GLOBAL_UNIQUE_IDENTIFIER: string     1399         534727                                                            534727                                               
                    |- LAST_EDITED_DATE: timestamp                                                                                                                                              
                    |- TAXONOMIC_ORDER: integer                                                                                                                                           

In [34]:
!bq rm -f {project_id}:ebird.ebird_small_5

In [35]:
!gcloud storage ls -l gs://c11-ebird/*.gz

1032617810  2023-11-22T14:27:03Z  gs://c11-ebird/data.nm.txt.gz
     60011  2023-11-22T17:52:45Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz
4815040900  2023-11-22T07:20:39Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
TOTAL: 3 objects, 5847718721 bytes (5.45GiB)


In [ ]:
# !gcloud storage rm gs://c11-ebird/*.gz

Removing objects:
Removing gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz...
Removing gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz...
Removing gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz...


## Transfer 5GB eBird sampling file

In [ ]:
%%bash
curl -s https://cnmi-ebird.sfo3.digitaloceanspaces.com/ebd_sampling_relSep-2023.tar |
dd bs=1M status=progress |
tar -tvf -


-rw-r--r-- ebdBuilder/ebdBuilder 4815040900 2023-10-15 18:45 ebd_sampling_relSep-2023.txt.gz
-rwxr-xr-x ebdBuilder/ebdBuilder       1960 2023-10-15 10:00 BCRCodes.txt
-rwxr-xr-x ebdBuilder/ebdBuilder     126957 2023-10-15 10:00 IBACodes.txt
-rwxr-xr-x ebdBuilder/ebdBuilder      39670 2023-10-15 10:00 USFWSCodes.txt
-rw-r--r-- ebdBuilder/ebdBuilder     431439 2023-10-15 10:00 eBird_Basic_Dataset_Metadata_v1.15.pdf
-rwxr-xr-x ebdBuilder/ebdBuilder        103 2023-10-15 10:00 recommended_citation.txt
-rwxr-xr-x ebdBuilder/ebdBuilder       6703 2023-10-15 10:00 terms_of_use.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4592M  100 4592M    0     0  46.4M      0  0:01:38  0:01:38 --:--:-- 46.9M

0+337897 records in
0+337897 records out
4815656960 bytes (4.8 GB, 4.5 GiB) copied, 98.8393 s, 48.7 MB/s


In [ ]:
%%bash
curl -s https://cnmi-ebird.sfo3.digitaloceanspaces.com/ebd_sampling_relSep-2023.tar |
tar -O -xf - ebd_sampling_relSep-2023.txt.gz |
dd bs=1M status=progress |
gcloud storage cp - gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz


Copying file://- to gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
  
4780635648 bytes (4.8 GB, 4.5 GiB) copied, 253 s, 18.9 MB/s
0+326225 records in
0+326225 records out
4815040900 bytes (4.8 GB, 4.5 GiB) copied, 253.737 s, 19.0 MB/s
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [36]:
!gcloud storage ls -l gs://c11-ebird/

1032617810  2023-11-22T14:27:03Z  gs://c11-ebird/data.nm.txt.gz
    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T17:52:45Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz
27103974002  2023-11-22T08:07:44Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt
4815040900  2023-11-22T07:20:39Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
TOTAL: 5 objects, 32952197421 bytes (30.69GiB)


In [ ]:
%%bash
gsutil cp gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz - |
gunzip |
dd bs=1M status=progress |
gsutil cp - gs://c11-ebird/ebd_sampling_relSep-2023.txt


Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]      0.0 B/s                                   
Operation completed over 1 objects.                                              


In [37]:
!gcloud storage ls -l gs://c11-ebird/

1032617810  2023-11-22T14:27:03Z  gs://c11-ebird/data.nm.txt.gz
    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T17:52:45Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz
27103974002  2023-11-22T08:07:44Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt
4815040900  2023-11-22T07:20:39Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
TOTAL: 5 objects, 32952197421 bytes (30.69GiB)


In [ ]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebd_sampling_relSep_2023 \
  gs://c11-ebird/ebd_sampling_relSep-2023.txt


Waiting on bqjob_r220523e18874cbcb_0000018bf7675d17_1 ... (15s) Current status: DONE   


In [38]:
!bq ls {project_id}:ebird

          tableId            Type    Labels   Time Partitioning   Clustered Fields  
 -------------------------- ------- -------- ------------------- ------------------ 
  ebd_NM_relSep_2023         TABLE                                                  
  ebd_sampling_relSep_2023   TABLE                                                  
  ebird_small                TABLE                                                  
  ebird_small_2              TABLE                                                  


In [ ]:
!bq show {project_id}:ebird.ebd_sampling_relSep_2023

Table default-256400:ebird.ebd_sampling_relSep_2023

   Last modified                   Schema                  Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- -------------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  22 Nov 14:20:43   |- LAST_EDITED_DATE: timestamp         111388197    27063315375                                                       27063315375           0                              
                    |- country: string                                                                                                                                                         
                    |- COUNTRY_CODE: string                                                                                                                                        

## New Mexico observations

In [ ]:
!curl -I https://cnmi-ebird.sfo3.digitaloceanspaces.com/data.nm.txt.gz

# 1,032,617,810

HTTP/2 200 
content-length: 1032617810
accept-ranges: bytes
last-modified: Thu, 16 Nov 2023 17:18:22 GMT
x-rgw-object-type: Normal
etag: "f65095637a2e54553346a337547e3b82-66"
x-amz-meta-s3cmd-attrs: atime:1700146076/ctime:1700146258/gid:0/gname:root/md5:c7ce3b4dbaf87bac3fca0f5d6d592f5c/mode:33188/mtime:1700146258/uid:0/uname:root
x-amz-request-id: tx000000aea1807f1f75728-00655e0f40-3c6f48c0-sfo3a
content-type: application/gzip
date: Wed, 22 Nov 2023 14:25:04 GMT
vary: Origin, Access-Control-Request-Headers, Access-Control-Request-Method
strict-transport-security: max-age=15552000; includeSubDomains; preload
x-envoy-upstream-healthchecked-cluster: 



In [ ]:
%%bash
curl -s https://cnmi-ebird.sfo3.digitaloceanspaces.com/data.nm.txt.gz |
dd bs=1M status=progress |
gcloud storage cp - gs://${GOOGLE_CLOUD_BUCKET}/data.nm.txt.gz


Copying file://- to gs://c11-ebird/data.nm.txt.gz
  
998744064 bytes (999 MB, 952 MiB) copied, 38 s, 26.3 MB/s
0+83921 records in
0+83921 records out
1032617810 bytes (1.0 GB, 985 MiB) copied, 38.3915 s, 26.9 MB/s
.............................................................................................................................................................................................

Average throughput: 27.7MiB/s


In [42]:
%%bash
gcloud storage ls -l gs://${GOOGLE_CLOUD_BUCKET}

1032617810  2023-11-22T14:27:03Z  gs://c11-ebird/data.nm.txt.gz
    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T17:52:45Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz
27103974002  2023-11-22T08:07:44Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt
4815040900  2023-11-22T07:20:39Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
TOTAL: 5 objects, 32952197421 bytes (30.69GiB)


In [ ]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebd_NM_relSep_2023 \
  gs://${GOOGLE_CLOUD_BUCKET}/data.nm.txt.gz


Waiting on bqjob_r8dae043b2db7705_0000018bf76f58ec_1 ... (194s) Current status: DONE   


In [ ]:
!bq ls {project_id}:ebird

          tableId            Type    Labels   Time Partitioning   Clustered Fields  
 -------------------------- ------- -------- ------------------- ------------------ 
  ebd_NM_relSep_2023         TABLE                                                  
  ebd_sampling_relSep_2023   TABLE                                                  
  ebird_small                TABLE                                                  


In [ ]:
!bq show {project_id}:ebird.ebd_NM_relSep_2023

Table default-256400:ebird.ebd_NM_relSep_2023

   Last modified                   Schema                   Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- --------------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  22 Nov 14:32:01   |- GLOBAL_UNIQUE_IDENTIFIER: string     10170370     4151731058                                                        4151731058                                           
                    |- LAST_EDITED_DATE: timestamp                                                                                                                                              
                    |- TAXONOMIC_ORDER: integer                                                                                                                                      

In [39]:
%%bigquery df --project {project_id}
select *
from ebird.ebd_NM_relSep_2023
limit 1000

Query is running:   0%|          |

Downloading:   0%|          |

In [40]:
df

,GLOBAL_UNIQUE_IDENTIFIER,LAST_EDITED_DATE,TAXONOMIC_ORDER,CATEGORY,TAXON_CONCEPT_ID,COMMON_NAME,SCIENTIFIC_NAME,SUBSPECIES_COMMON_NAME,SUBSPECIES_SCIENTIFIC_NAME,EXOTIC_CODE,...,EFFORT_AREA_HA,NUMBER_OBSERVERS,ALL_SPECIES_REPORTED,GROUP_IDENTIFIER,HAS_MEDIA,APPROVED,REVIEWED,REASON,TRIP_COMMENTS,SPECIES_COMMENTS
0,URN:CornellLabOfOrnithology:EBIRD:OBS22089322,2021-03-26 05:56:39.420466+00:00,33853,species,avibase-7C2FCB13,Rose-breasted Grosbeak,Pheucticus ludovicianus,None,None,None,...,NaN,<NA>,1,None,0,1,0,None,None,None
1,URN:CornellLabOfOrnithology:EBIRD:OBS1352217352,2022-02-25 16:11:27.244439+00:00,21054,species,avibase-69544B59,American Crow,Corvus brachyrhynchos,None,None,None,...,NaN,1,0,None,0,1,0,None,None,None
2,URN:CornellLabOfOrnithology:EBIRD:OBS33720336,2021-03-26 05:57:55.198592+00:00,26546,species,avibase-A15C5071,Blue-gray Gnatcatcher,Polioptila caerulea,None,None,None,...,NaN,<NA>,1,None,0,1,0,None,None,None
3,URN:CornellLabOfOrnithology:EBIRD:OBS262951408,2020-04-11 17:42:55+00:00,23610,species,avibase-58C502EA,Barn Swallow,Hirundo rustica,None,None,None,...,NaN,2,1,G951347,0,1,0,None,None,None
4,URN:CornellLabOfOrnithology:EBIRD:OBS553681096,2017-12-01 09:51:07+00:00,31648,species,avibase-89431E9F,House Finch,Haemorhous mexicanus,None,None,None,...,NaN,2,0,None,0,1,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,URN:CornellLabOfOrnithology:EBIRD:OBS42623066,2018-11-07 15:06:08+00:00,11110,species,avibase-9D06AF89,Hairy Woodpecker,Dryobates villosus,None,None,None,...,NaN,3,1,None,0,1,0,None,None,None
996,URN:CornellLabOfOrnithology:EBIRD:OBS551225513,2021-01-10 09:49:23.659195+00:00,8062,species,avibase-420850D6,Northern Goshawk,Accipiter gentilis,None,None,None,...,NaN,1,0,None,0,1,0,None,None,None
997,URN:CornellLabOfOrnithology:EBIRD:OBS1724503013,2023-05-13 22:22:33.446387+00:00,33855,species,avibase-824361E5,Black-headed Grosbeak,Pheucticus melanocephalus,None,None,None,...,NaN,1,0,None,0,1,0,None,None,None
998,URN:CornellLabOfOrnithology:EBIRD:OBS263555463,2014-07-13 12:31:24+00:00,33188,species,avibase-5CBA3391,Great-tailed Grackle,Quiscalus mexicanus,None,None,None,...,NaN,1,0,None,0,1,0,None,None,None
